In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pickle

import numpy as np
from scipy import interpolate

sys.path.append(os.environ['DM21CM_DIR'])
import dm21cm.physics as phys

sys.path.append(os.environ['DH_DIR'])
from darkhistory.main import evolve as dh_evolve
import darkhistory.physics as dhphys

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../matplotlibrc')

## 1. DarkHistory runs

In [2]:
base_21cmfast = np.load(f'../data/run_info/base_records.npy', allow_pickle=True).item()

start_rs = base_21cmfast['z'][0] + 1
end_rs = base_21cmfast['z'][-1] + 1

In [3]:
base_21cmfast['z'][1] + 1

40.54271295180308

### 1.1 regular run

In [4]:
from darkhistory.config import load_data

In [5]:
load_data('dep_tf', prefix='/n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_test')

****** Loading transfer functions... ******
Using data at /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_test
    for propagating photons...   Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******


{'highengphot': <darkhistory.spec.transferfunclist.TransferFuncInterp at 0x7fa84288e9e0>,
 'lowengphot': <darkhistory.spec.transferfunclist.TransferFuncInterp at 0x7fa84288cbb0>,
 'lowengelec': <darkhistory.spec.transferfunclist.TransferFuncInterp at 0x7fa84288f9a0>,
 'highengdep': <darkhistory.history.histools.IonRSInterp at 0x7fa84288c760>,
 'CMB_engloss': <darkhistory.history.histools.IonRSInterp at 0x7fa81092d150>}

In [6]:
run_name = 'phph'

soln = dh_evolve(
    DM_process='swave', mDM=1e10, sigmav=1e-23, primary='phot_delta',
    start_rs=3000, end_rs=5,
    coarsen_factor=12, backreaction=True, 
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    reion_switch=False,
    use_tqdm=True, verbose=1,
    custom_dlnz=True
)

****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******
Loading time: 0.844 s


  0%|                                                                                       | 0/534 [00:00<?, ?it/s]

Initialization time: 0.067 s


 10%|███████▉                                                                      | 54/534 [00:07<01:05,  7.31it/s]/n/home07/yitians/.local/lib/python3.10/site-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
 64%|█████████████████████████████████████████████████▌                           | 344/534 [00:48<00:26,  7.24it/s]WARNING:root:Overwriting coarsen_factor and dlnz!
573it [00:53, 10.68it/s]                                                                                            

Main loop time: 53.632 s


In [7]:
pickle.dump(soln, open(f'../data/run_info/DH/{run_name}_21tf_soln.p', 'wb'))

### 1.2 force case A with fake reionization

In [8]:
run_name = 'phph_caseA_in_reion'

soln = dh_evolve(
    DM_process='swave', mDM=1e10, sigmav=1e-23, primary='phot_delta',
    start_rs=3000, end_rs=6,
    coarsen_factor=12, backreaction=True, 
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    reion_switch=True, reion_rs=base_21cmfast['z'][1]+1,
    photoion_rate_func = [lambda rs: 0.] * 3,
    photoheat_rate_func = [lambda rs: 0.] * 3,
    use_tqdm=True, verbose=1
)

****** Loading transfer functions... ******
Using data at /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1
    for propagating photons...   Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******
****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******
Loading time: 32.039 s


  0%|                                                                                           | 0/518 [00:00<?, ?it/s]

Initialization time: 0.127 s


 10%|████████▌                                                                         | 54/518 [00:06<00:59,  7.83it/s]/n/home07/yitians/.local/lib/python3.10/site-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
100%|█████████████████████████████████████████████████████████████████████████████████| 518/518 [01:17<00:00,  6.72it/s]


Main loop time: 77.095 s


In [9]:
pickle.dump(soln, open(f'../../data/run_info/DH/{run_name}_soln.p', 'wb'))

### 1.3 custom injection

In [ ]:
import darkhistory.physics as dhphys
from darkhistory.spec import pppc
from config import load_data

In [ ]:
# custom xe levels
def x_reion_func(rs):
    # extrapolate z low so that darkhistory doesn't complain
    z_arr = base_records['z'].copy()
    z_arr = np.append(z_arr, 2 * z_arr[-1] - z_arr[-2])
    x_e_arr = base_records['x_e'].copy()
    x_e_arr = np.append(x_e_arr, x_e_arr[-1])
    return interpolate.interp1d(1 + z_arr, x_e_arr)(rs)

In [21]:
run_name = 'phph_turnon_z35_comp'

mDM = 1e10
sigmav = 1e-23
struct_boost = phys.struct_boost_func('erfc 1e-3')

photeng = load_data('binning')['phot']
eleceng = load_data('binning')['elec']

def in_spec_elec_func(rs):
    in_spec_elec = pppc.get_pppc_spec(1e10, eleceng, 'phot_delta', 'elec')
    in_spec_elec.rs = rs
    if rs > 36:
        in_spec_elec *= 0
    return in_spec_elec

def in_spec_phot_func(rs):
    in_spec_phot = pppc.get_pppc_spec(1e10, photeng, 'phot_delta', 'phot')
    in_spec_phot.rs = rs
    if rs > 36:
        in_spec_phot *= 0
    return in_spec_phot

def rate_func_N(rs):
    return (
        dhphys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)
        * struct_boost(rs) / (2*mDM)
    )
def rate_func_eng(rs):
    return (
        dhphys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav) 
        * struct_boost(rs)
    )

In [25]:
soln = main.evolve(
    in_spec_elec = in_spec_elec_func,
    in_spec_phot = in_spec_phot_func,
    rate_func_N = rate_func_N,
    rate_func_eng = rate_func_eng,
    start_rs=3000, end_rs=7,
    coarsen_factor=12, backreaction=True, reion_switch=True,
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    verbose=1,
    use_tqdm=False
)

Loading time: 0.000 s
Initialization time: 0.004 s
Main loop time: 49.761 s


### 1.1 Get recfast values

In [20]:
run_name = 'base_recfast'

rs_s = np.geomspace(start_rs, end_rs, 100)
x_e_s = dhphys.xHII_std(rs_s)
T_k_s = dhphys.Tm_std(rs_s) / phys.kB

arr_records = {
    'z' : rs_s - 1,
    'x_e' : x_e_s,
    'T_k' : T_k_s,
}
print(run_name)

base_recfast


In [22]:
np.save(f'../../data/run_info/DH/{run_name}_records', arr_records)

## 2. Make arr_records and save

In [9]:
run_name = 'phph_21tf'
soln = pickle.load(open(f'../data/run_info/DH/{run_name}_soln.p', 'rb'))

In [10]:
arr_records = {
    'z' : soln['rs'] - 1,
    'x_e' : soln['x'][:,0],
    'T_k' : soln['Tm']/phys.kB,
    'f_heat' : soln['f']['low']['heat'] + soln['f']['high']['heat'],
    'f_ion' : soln['f']['low']['H ion']  + soln['f']['high']['H ion'] + \
              soln['f']['low']['He ion'] + soln['f']['high']['He ion'],
    'f_exc' : soln['f']['low']['exc'] + soln['f']['high']['exc'],
}

z_low, z_high = 6, 44
z_arr = arr_records['z'][::-1]
i_low = np.searchsorted(z_arr, z_low)
i_high = np.searchsorted(z_arr, z_high)

for k in arr_records.keys():
    arr_records[k] = arr_records[k][::-1][i_low:i_high]
    
print(run_name)

phph_21tf


In [12]:
np.save(f'../data/run_info/DH/{run_name}_records', arr_records)